# Create the database schema

### Import necessory library and build functions for the tasks

In [1]:

from functools import wraps
from sqlalchemy.exc import OperationalError, SQLAlchemyError
from main.database_utils import DatabaseConnector
from sqlalchemy import text, create_engine, MetaData, Table


In [2]:
class SchemaUpdater(DatabaseConnector):
    """
    A class to update the schema of a table in a database based on desired changes.

    Inherits from:
    -------------
    DatabaseConnector:
        A base class to handle database connections.

    Attributes:
    -----------
    creds : dict
        Database credentials obtained from the credentials file.
    engine : sqlalchemy.engine.Engine
        SQLAlchemy engine initialized from the credentials.
    schema : dict
        The current schema of the table, with column names as keys and their data types as values.

    Methods:
    --------
    get_schema(connection, table_name):
        Retrieves the schema for a specific table.
    
    update_schema(connection, table_name, desired_schema):
        Updates the schema of a given table based on a desired schema dictionary.
    """
    def __init__(self, target_creds_path="target_db_creds.yaml"):
    # def __init__(self, creds_path="db_creds.yaml", target_creds_path="target_db_creds.yaml"):
        super().__init__(target_creds_path)
        self.creds = self.read_db_creds(target_creds_path)

    def get_schema(self, connection, table_name):
        metadata = MetaData()
        table = Table(table_name, metadata, autoload_with=connection)
        self.schema = {column.name: str(column.type) for column in table.columns}

    def update_schema(self, connection, table_name, desired_schema):
        """
        Updates the schema of the specified table based on the desired schema.

        Parameters:
        ----------
        connection : sqlalchemy.engine.Connection
            The connection to the database.
        table_name : str
            The name of the table to update.
        desired_schema : dict
            A dictionary containing the desired schema for the table.
        """
        # Retrieve the current schema for the table
        self.get_schema(connection, table_name)
        print(f"Original schema for table {table_name}: {self.schema}")

        # Compare and prepare schema changes if necessary
        alter_statements = []
        for column, desired_type in desired_schema.items():
            current_type = self.schema.get(column)
            print(f"Checking column '{column}': current type is '{current_type}', desired type is '{desired_type}'")

            if current_type != desired_type:
                alter_statements.append(
                    f"ALTER COLUMN {column} SET DATA TYPE {desired_type} USING {column}::{desired_type}"
                )

        if alter_statements:
            sql_command = f"ALTER TABLE {table_name} {', '.join(alter_statements)};"
            print(f"Executing SQL: {sql_command}")

            try:
                connection.execute(text(sql_command))
                connection.commit() 
                print(f"Schema updates applied successfully for {table_name}.")
                self.get_schema(connection, table_name)
                print(f"Current schema for table {table_name}: {self.schema}")
            except Exception as e:
                print(f"Error applying schema updates for {table_name}: {e}")
        else:
            print(f"No additional schema changes needed for {table_name}.")

def get_max_lengths(connection, table_name, columns):
    """
    Retrieves the maximum length of data in each specified column of a given table.

    Parameters:
    ----------
    engine : sqlalchemy.engine.Engine
        The SQLAlchemy engine used to connect to the database.
    table_name : str
        The name of the table to query.
    columns : list of str
        A list of column names for which to calculate the maximum data length.

    Returns:
    -------
    dict:
        A dictionary where each key is a column name and its corresponding value is the 
        maximum length of the data in that column. If the column has no data, the length is 0.
    
    Example:
    -------
    >>> max_lengths = get_max_lengths(engine, 'products_table', ['product_name', 'category'])
    >>> print(max_lengths)
    {'product_name': 50, 'category': 30}
    """
    max_lengths = {}
    for column in columns:
        result = connection.execute(text(f"""
            SELECT MAX(LENGTH({column})) AS max_length
            FROM {table_name}
            WHERE {column} IS NOT NULL;
        """))
        max_length = result.scalar() or 0  # Default to 0 if no data
        max_lengths[column] = max_length
    return max_lengths

In [3]:
# Instantiate SchemaUpdater
su = SchemaUpdater()
# Read target credentials
su.creds = su.read_db_creds(su.target_creds_path)
# Create engine
su.engine = create_engine(
    f"postgresql://{su.creds['RDS_USER']}:{su.creds['RDS_PASSWORD']}@{su.creds['RDS_HOST']}:{su.creds['RDS_PORT']}/{su.creds['RDS_DATABASE']}"
)
# Using the SchemaUpdater

### Task 1. Change the data types of columns in "orders_table" to correspond to those seen in the table below.
```
+------------------+--------------------+--------------------+
|   orders_table   | current data type  | required data type |
+------------------+--------------------+--------------------+
| date_uuid        | TEXT               | UUID               |
| user_uuid        | TEXT               | UUID               |
| card_number      | TEXT               | VARCHAR(?)         |
| store_code       | TEXT               | VARCHAR(?)         |
| product_code     | TEXT               | VARCHAR(?)         |
| product_quantity | BIGINT             | SMALLINT           |
+------------------+--------------------+--------------------+
```

The ? in VARCHAR should be replaced with an integer representing the maximum length of the values in that column.


In [4]:

# Assuming you already have your connection and table name
columns_to_check = ['card_number', 'store_code', 'product_code']
# max_lengths = get_max_lengths(connection, 'orders_table', columns_to_check)
# Define the desired schema for the table


# build connection with database
with su.engine.connect() as connection:
    # max_lengths = get_max_lengths(connection, 'orders_table', columns_to_check)
    max_lengths = get_max_lengths(connection, 'orders_table', columns_to_check)
    # Define the desired schema for the table
    desired_orders_table_schema = {
        'user_uuid': 'UUID',
        'date_uuid': 'UUID',
        'card_number': f'VARCHAR({max_lengths["card_number"]})',
        'store_code': f'VARCHAR({max_lengths["store_code"]})',
        'product_code': f'VARCHAR({max_lengths["product_code"]})',
        'product_quantity': 'SMALLINT'
    }
    # Alter the specific table schema in that database
    su.update_schema(connection, 'orders_table', desired_orders_table_schema)


Original schema for table orders_table: {'level_0': 'BIGINT', 'index': 'BIGINT', 'date_uuid': 'TEXT', 'user_uuid': 'TEXT', 'card_number': 'TEXT', 'store_code': 'TEXT', 'product_code': 'TEXT', 'product_quantity': 'TEXT'}
Checking column 'user_uuid': current type is 'TEXT', desired type is 'UUID'
Checking column 'date_uuid': current type is 'TEXT', desired type is 'UUID'
Checking column 'card_number': current type is 'TEXT', desired type is 'VARCHAR(19)'
Checking column 'store_code': current type is 'TEXT', desired type is 'VARCHAR(12)'
Checking column 'product_code': current type is 'TEXT', desired type is 'VARCHAR(11)'
Checking column 'product_quantity': current type is 'TEXT', desired type is 'SMALLINT'
Executing SQL: ALTER TABLE orders_table ALTER COLUMN user_uuid SET DATA TYPE UUID USING user_uuid::UUID, ALTER COLUMN date_uuid SET DATA TYPE UUID USING date_uuid::UUID, ALTER COLUMN card_number SET DATA TYPE VARCHAR(19) USING card_number::VARCHAR(19), ALTER COLUMN store_code SET DATA 

### Task 2. Re-schema "dim_users" as seen in the table below.

```
The column required to be changed in the users table are as follows:

+----------------+--------------------+--------------------+
| dim_users      | current data type  | required data type |
+----------------+--------------------+--------------------+
| first_name     | TEXT               | VARCHAR(255)       |
| last_name      | TEXT               | VARCHAR(255)       |
| date_of_birth  | TEXT               | DATE               |
| country_code   | TEXT               | VARCHAR(?)         |
| user_uuid      | TEXT               | UUID               |
| join_date      | TEXT               | DATE               |
+----------------+--------------------+--------------------+
```

In [5]:


# build connection with database
with su.engine.connect() as connection:
    columns_to_check = ["country_code"]
    max_lengths = get_max_lengths(connection, 'dim_users', columns_to_check)
    # Define the desired schema for the table
    desired_users_table_schema = {
        'first_name': 'VARCHAR(255)',
        'last_name': 'VARCHAR(255)',
        'date_of_birth': 'DATE',
        'country_code': f'VARCHAR({max_lengths["country_code"]})',
        'user_uuid': 'UUID',
        'join_date': 'DATE'
    }
    # Alter the specific table schema in that database
    su.update_schema(connection, 'dim_users', desired_users_table_schema)


Original schema for table dim_users: {'index': 'BIGINT', 'first_name': 'TEXT', 'last_name': 'TEXT', 'date_of_birth': 'DATE', 'company': 'TEXT', 'email_address': 'TEXT', 'address': 'TEXT', 'country': 'TEXT', 'country_code': 'TEXT', 'phone_number': 'TEXT', 'join_date': 'DATE', 'user_uuid': 'TEXT'}
Checking column 'first_name': current type is 'TEXT', desired type is 'VARCHAR(255)'
Checking column 'last_name': current type is 'TEXT', desired type is 'VARCHAR(255)'
Checking column 'date_of_birth': current type is 'DATE', desired type is 'DATE'
Checking column 'country_code': current type is 'TEXT', desired type is 'VARCHAR(2)'
Checking column 'user_uuid': current type is 'TEXT', desired type is 'UUID'
Checking column 'join_date': current type is 'DATE', desired type is 'DATE'
Executing SQL: ALTER TABLE dim_users ALTER COLUMN first_name SET DATA TYPE VARCHAR(255) USING first_name::VARCHAR(255), ALTER COLUMN last_name SET DATA TYPE VARCHAR(255) USING last_name::VARCHAR(255), ALTER COLUMN cou

### Task 3. Re-schema "dim_store_details" as seen in the table below.
There are two latitude columns in the store details table. Using SQL, merge one of the columns into the other so you have one latitude column.

Then set the data types for each column as shown below:
```
+---------------------+-------------------+------------------------+
| store_details_table | current data type |   required data type   |
+---------------------+-------------------+------------------------+
| longitude           | TEXT              | FLOAT                  |
| locality            | TEXT              | VARCHAR(255)           |
| store_code          | TEXT              | VARCHAR(?)             |
| staff_numbers       | TEXT              | SMALLINT               |
| opening_date        | TEXT              | DATE                   |
| store_type          | TEXT              | VARCHAR(255) NULLABLE  |
| latitude            | TEXT              | FLOAT                  |
| country_code        | TEXT              | VARCHAR(?)             |
| continent           | TEXT              | VARCHAR(255)           |
+---------------------+-------------------+------------------------+
```
There is a row that represents the business's website change the location column values from N/A to NULL.

In [6]:

cols = ['store_code']
# build connection with database
with su.engine.connect() as connection:
    # There is one store_code which mismatch in order_table and dim_store_details
    # add a row in dim_store_details keep the data in order_table as it is valid
    # # Define  SQL insert statement
    # insert_sql = """
    # INSERT INTO dim_store_details 
    # (address, longitude, latitude, locality, store_code, staff_numbers, opening_date, store_type, country_code, continent) 
    # VALUES ('Unknown', 0.0, 0.0, 'Unknown', 'WEB-1388012W', 0, NULL, 'Unknown', 'Unknown', 'Unknown');
    # """
    with connection.begin():
        # connection.execute(text(insert_sql))
        # print("Record inserted successfully.")
        max_lengths = get_max_lengths(connection, "dim_store_details", cols)
        # Define the desired schema for the table
        desired_store_details_schema = {
            'longitude': 'FLOAT',
            'locality': 'VARCHAR(255)',
            'store_code': f'VARCHAR({max_lengths["store_code"]})',  
            'staff_numbers': 'SMALLINT',
            'opening_date': 'DATE',
            'store_type': 'VARCHAR(255)',  
            'latitude': 'FLOAT',
            'country_code': 'VARCHAR(10)', 
            'continent': 'VARCHAR(255)'
        }

    # Alter the specific table schema in that database
    su.update_schema(connection, 'dim_store_details', desired_store_details_schema)

Original schema for table dim_store_details: {'index': 'BIGINT', 'address': 'TEXT', 'longitude': 'DOUBLE PRECISION', 'lat': 'TEXT', 'locality': 'TEXT', 'store_code': 'TEXT', 'staff_numbers': 'BIGINT', 'opening_date': 'DATE', 'store_type': 'TEXT', 'latitude': 'DOUBLE PRECISION', 'country_code': 'TEXT', 'continent': 'TEXT'}
Checking column 'longitude': current type is 'DOUBLE PRECISION', desired type is 'FLOAT'
Checking column 'locality': current type is 'TEXT', desired type is 'VARCHAR(255)'
Checking column 'store_code': current type is 'TEXT', desired type is 'VARCHAR(12)'
Checking column 'staff_numbers': current type is 'BIGINT', desired type is 'SMALLINT'
Checking column 'opening_date': current type is 'DATE', desired type is 'DATE'
Checking column 'store_type': current type is 'TEXT', desired type is 'VARCHAR(255)'
Checking column 'latitude': current type is 'DOUBLE PRECISION', desired type is 'FLOAT'
Checking column 'country_code': current type is 'TEXT', desired type is 'VARCHAR(1

### Task 4. Re-schema "dim_products" as seen in the table below.

You will need to do some work on the products table before casting the data types correctly.

The product_price column has a £ character which you need to remove using SQL.

The team that handles the deliveries would like a new human-readable column added for the weight so they can quickly make decisions on delivery weights.

Add a new column weight_class which will contain human-readable values based on the weight range of the product.
```
+--------------------------+-------------------+
| weight_class VARCHAR(?)  | weight range(kg)  |
+--------------------------+-------------------+
| Light                    | < 2               |
| Mid_Sized                | >= 2 - < 40       |
| Heavy                    | >= 40 - < 140     |
| Truck_Required           | => 140            |
+----------------------------+-----------------+
```

In [7]:

# Build connection with the database
with su.engine.connect() as connection:
    cols_to_check = ['product_code']
    max_lengths = get_max_lengths(connection, "dim_products", cols_to_check)

    # Step 1: Alter the weight_class column to VARCHAR(20) to handle larger values
    alter_weight_class_column_query = text("""
        ALTER TABLE dim_products 
        ADD COLUMN weight_class VARCHAR(20);                                   
    """)
    connection.execute(alter_weight_class_column_query)

    # Step 2: Populate the weight_class column based on the weight range
    update_weight_class_query = text("""
    UPDATE dim_products
    SET weight_class = 
        CASE
            WHEN weight < 2 THEN 'Light'
            WHEN weight >= 2 AND weight < 40 THEN 'Mid_Sized'
            WHEN weight >= 40 AND weight < 140 THEN 'Heavy'
            WHEN weight >= 140 THEN 'Truck_Required'
        END;
    """)
    
    connection.execute(update_weight_class_query)

    # Step 2: Define the desired schema for the products_table (excluding weight_class as it's newly added)
    desired_products_table_schema = {
        'product_name': 'VARCHAR(255)',
        'product_price': 'FLOAT',
        'weight': 'FLOAT',
        'category': 'VARCHAR(255)',
        'date_added': 'DATE',
        'uuid': 'UUID',
        'removed': 'VARCHAR(20)',
        'product_code': f'VARCHAR({max_lengths["product_code"]})'
    }

    # Alter the schema in the products_table (excluding weight_class)
    su.update_schema(connection, 'dim_products', desired_products_table_schema)
    

Original schema for table dim_products: {'product_name': 'TEXT', 'product_price': 'DOUBLE PRECISION', 'weight': 'DOUBLE PRECISION', 'category': 'TEXT', 'EAN': 'TEXT', 'date_added': 'DATE', 'uuid': 'TEXT', 'removed': 'TEXT', 'product_code': 'TEXT', 'weight_class': 'VARCHAR(20)'}
Checking column 'product_name': current type is 'TEXT', desired type is 'VARCHAR(255)'
Checking column 'product_price': current type is 'DOUBLE PRECISION', desired type is 'FLOAT'
Checking column 'weight': current type is 'DOUBLE PRECISION', desired type is 'FLOAT'
Checking column 'category': current type is 'TEXT', desired type is 'VARCHAR(255)'
Checking column 'date_added': current type is 'DATE', desired type is 'DATE'
Checking column 'uuid': current type is 'TEXT', desired type is 'UUID'
Checking column 'removed': current type is 'TEXT', desired type is 'VARCHAR(20)'
Checking column 'product_code': current type is 'TEXT', desired type is 'VARCHAR(11)'
Executing SQL: ALTER TABLE dim_products ALTER COLUMN prod

### Task 5. Re-schema "dim_date_times" as seen in the table below.

Now update the date table with the correct types:
```
+-----------------+-------------------+--------------------+
| dim_date_times  | current data type | required data type |
+-----------------+-------------------+--------------------+
| month           | TEXT              | VARCHAR(?)         |
| year            | TEXT              | VARCHAR(?)         |
| day             | TEXT              | VARCHAR(?)         |
| time_period     | TEXT              | VARCHAR(?)         |
| date_uuid       | TEXT              | UUID               |
+-----------------+-------------------+--------------------+
```

In [8]:
# Build connection with the database
with su.engine.connect() as connection:
    # Step 1: Define the desired schema for the dim_date_time table
    desired_dim_date_time_schema = {
        'month': 'VARCHAR(50)',  # Adjust length as necessary
        'year': 'VARCHAR(50)',   # Adjust length as necessary
        'day': 'VARCHAR(50)',     # Adjust length as necessary
        'time_period': 'VARCHAR(50)',  # Adjust length as necessary
        'date_uuid': 'UUID'
    }

    # Alter the schema in the dim_date_time table
    su.update_schema(connection, 'dim_date_times', desired_dim_date_time_schema)

Original schema for table dim_date_times: {'timestamp': 'TEXT', 'month': 'BIGINT', 'year': 'BIGINT', 'day': 'BIGINT', 'time_period': 'TEXT', 'date_uuid': 'TEXT'}
Checking column 'month': current type is 'BIGINT', desired type is 'VARCHAR(50)'
Checking column 'year': current type is 'BIGINT', desired type is 'VARCHAR(50)'
Checking column 'day': current type is 'BIGINT', desired type is 'VARCHAR(50)'
Checking column 'time_period': current type is 'TEXT', desired type is 'VARCHAR(50)'
Checking column 'date_uuid': current type is 'TEXT', desired type is 'UUID'
Executing SQL: ALTER TABLE dim_date_times ALTER COLUMN month SET DATA TYPE VARCHAR(50) USING month::VARCHAR(50), ALTER COLUMN year SET DATA TYPE VARCHAR(50) USING year::VARCHAR(50), ALTER COLUMN day SET DATA TYPE VARCHAR(50) USING day::VARCHAR(50), ALTER COLUMN time_period SET DATA TYPE VARCHAR(50) USING time_period::VARCHAR(50), ALTER COLUMN date_uuid SET DATA TYPE UUID USING date_uuid::UUID;
Schema updates applied successfully for 

### Task 6. Re-schema "dim_card_details" as seen in the table below.

Now we need to update the last table for the card details.

Make the associated changes after finding out what the lengths of each variable should be:
```
+------------------------+-------------------+--------------------+
|    dim_card_details    | current data type | required data type |
+------------------------+-------------------+--------------------+
| card_number            | TEXT              | VARCHAR(?)         |
| expiry_date            | TEXT              | VARCHAR(?)         |
| date_payment_confirmed | TEXT              | DATE               |
+------------------------+-------------------+--------------------+
```

In [9]:

# Build connection with the database
with su.engine.connect() as connection:
    cols_to_check = ['card_number']
    max_lengths = get_max_lengths(connection, "dim_card_details", cols_to_check)
    # Step 1: Define the desired schema for the dim_card_details table
    desired_dim_card_details_schema = {
        'card_number': f'VARCHAR({max_lengths["card_number"]})',  # Adjust length based on your requirements (max for card numbers)
        'expiry_date': 'VARCHAR(7)',    # Format: MM/YYYY, hence 7 characters
        'date_payment_confirmed': 'DATE'
    }

    # Alter the schema in the dim_card_details table
    su.update_schema(connection, 'dim_card_details', desired_dim_card_details_schema)

Original schema for table dim_card_details: {'card_number': 'TEXT', 'expiry_date': 'TEXT', 'card_provider': 'TEXT', 'date_payment_confirmed': 'DATE'}
Checking column 'card_number': current type is 'TEXT', desired type is 'VARCHAR(19)'
Checking column 'expiry_date': current type is 'TEXT', desired type is 'VARCHAR(7)'
Checking column 'date_payment_confirmed': current type is 'DATE', desired type is 'DATE'
Executing SQL: ALTER TABLE dim_card_details ALTER COLUMN card_number SET DATA TYPE VARCHAR(19) USING card_number::VARCHAR(19), ALTER COLUMN expiry_date SET DATA TYPE VARCHAR(7) USING expiry_date::VARCHAR(7);
Schema updates applied successfully for dim_card_details.
Current schema for table dim_card_details: {'card_number': 'VARCHAR(19)', 'expiry_date': 'VARCHAR(7)', 'card_provider': 'TEXT', 'date_payment_confirmed': 'DATE'}


### Task 7. Set primary keys: 
Adding the primary keys to each of the tables prefixed with dim.

Each table will serve the orders_table which will be the single source of truth for our orders.

Check the column header of the orders_table you will see all but one of the columns exist in one of our tables prefixed with dim.

We need to update the columns in the dim tables with a primary key that matches the same column in the orders_table.

Using SQL, update the respective columns as primary key columns.



In [10]:

# Define a function to add primary key constraints
def add_primary_key(connection, table_name, column_name):
    # Use the alter statement to add primary key

    try:
        sql_comm = f"""
        ALTER TABLE {table_name} ADD PRIMARY KEY ({column_name});
        """
        connection.execute(text(sql_comm))
        print(text(sql_comm))
        
        print()
        
    except Exception as e:
        print(f"Error adding primary key for {table_name}: {e}")

In [11]:
# primary key and table dictionary
primary_keys_dict = {'dim_date_times': 'date_uuid', 'dim_users': 'user_uuid', 'dim_card_details': 'card_number', 'dim_store_details': 'store_code', 'dim_products': 'product_code'}
# Set primary keys for each dim table
with su.engine.connect() as connection:
    # Start a transaction
    with connection.begin():
        for py_key in primary_keys_dict.keys():
            try:
                add_primary_key(connection, table_name=py_key, column_name=primary_keys_dict[py_key])
            except Exception as e:
                print(f"Error occurred: {e}")





        ALTER TABLE dim_date_times ADD PRIMARY KEY (date_uuid);
        


        ALTER TABLE dim_users ADD PRIMARY KEY (user_uuid);
        


        ALTER TABLE dim_card_details ADD PRIMARY KEY (card_number);
        


        ALTER TABLE dim_store_details ADD PRIMARY KEY (store_code);
        


        ALTER TABLE dim_products ADD PRIMARY KEY (product_code);
        



In [12]:
# Check a table and confirm primary key:
def check_primary_key(table_name):
    query = f"""
    SELECT
        kcu.column_name
    FROM
        information_schema.table_constraints AS tc
    JOIN
        information_schema.key_column_usage AS kcu
        ON kcu.constraint_name = tc.constraint_name
    WHERE
        tc.table_name = '{table_name}'
        AND tc.constraint_type = 'PRIMARY KEY';
    """

    with su.engine.connect() as connection:
        result = connection.execute(text(query))
        primary_keys = [row[0] for row in result]
        
    return primary_keys

# Example usage
for table in primary_keys_dict.keys():
    primary_keys = check_primary_key(table)
    print(f"Primary keys for {table}: {primary_keys}")

Primary keys for dim_date_times: ['date_uuid']
Primary keys for dim_users: ['user_uuid']
Primary keys for dim_card_details: ['card_number']
Primary keys for dim_store_details: ['store_code']
Primary keys for dim_products: ['product_code']


### Task 8. create foreign keys:

Create the foreign keys in the orders_table to reference the primary keys in the other tables.

Use SQL to create those foreign key constraints that reference the primary keys of the other table.

This makes the star-based database schema complete.

In [13]:
# Define a function to add foreign key constraints
def add_foreign_key(connection, table_name, column_name, ref_table, ref_column):
    try:
        sql_comm = f"""
        ALTER TABLE {table_name}
        ADD CONSTRAINT fk_{table_name}_{column_name}
        FOREIGN KEY ({column_name})
        REFERENCES {ref_table}({ref_column});
        """
        connection.execute(text(sql_comm))
        print(text(sql_comm))
    except Exception as e:
        print(f"Error adding foreign key for {table_name}: {e}")

# Define the relationships between orders_table and dim tables
foreign_keys_dict = {
    'dim_date_times': 'date_uuid',
    'dim_users': 'user_uuid',
    'dim_card_details': 'card_number',
    'dim_store_details': 'store_code',
    'dim_products': 'product_code'
}

# Apply foreign key constraints to orders_table
with su.engine.connect() as connection:
    for ref_table, ref_column in foreign_keys_dict.items():
        try:
            add_foreign_key(
                connection, 
                table_name='orders_table', 
                column_name=ref_column, 
                ref_table=ref_table, 
                ref_column=ref_column
            )
        except Exception as e:
            print(f"Error occurred: {e}")


        ALTER TABLE orders_table
        ADD CONSTRAINT fk_orders_table_date_uuid
        FOREIGN KEY (date_uuid)
        REFERENCES dim_date_times(date_uuid);
        

        ALTER TABLE orders_table
        ADD CONSTRAINT fk_orders_table_user_uuid
        FOREIGN KEY (user_uuid)
        REFERENCES dim_users(user_uuid);
        

        ALTER TABLE orders_table
        ADD CONSTRAINT fk_orders_table_card_number
        FOREIGN KEY (card_number)
        REFERENCES dim_card_details(card_number);
        

        ALTER TABLE orders_table
        ADD CONSTRAINT fk_orders_table_store_code
        FOREIGN KEY (store_code)
        REFERENCES dim_store_details(store_code);
        

        ALTER TABLE orders_table
        ADD CONSTRAINT fk_orders_table_product_code
        FOREIGN KEY (product_code)
        REFERENCES dim_products(product_code);
        
